## **Database Connection and Data Loading**

In [18]:
import xlrd
import mysql.connector
import pandas as pd

conexion = mysql.connector.connect(
    host=" *** ",
    user=" *** ",
    password=" *** ",
    database= " *** "
)

### 4 variables are defined, which are the name of the database, as well as the name of the 3 tables

In [26]:
base_datos = "sandbox"
tabla_ventas = "ventas"
tabla_calendario = "calendario"
tabla_pedidos = "pedidos"

cursor = conexion.cursor()
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {base_datos}")

cursor.execute(f"USE {base_datos}")

### The 3 tables are created, specifying the columns of each with their data types and constraints

In [27]:
cursor.execute(f"DROP TABLE IF EXISTS {tabla_ventas}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {tabla_ventas} ("
               "FAMILIA VARCHAR(255),"
               "Tipo VARCHAR(255),"
               "FechaVenta DATE,"
               "HoraVenta INT,"
               "Articulo VARCHAR(255),"
               "Cantidad FLOAT,"
               "Precio FLOAT,"
               "Importe FLOAT"
               ")")

In [28]:
cursor.execute(f"DROP TABLE IF EXISTS {tabla_calendario}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {tabla_calendario} ("
               "Fecha DATE,"
               "Festivo VARCHAR(255)"
               ")")

In [29]:
cursor.execute(f"DROP TABLE IF EXISTS {tabla_pedidos}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {tabla_pedidos} ("
               "Tipo VARCHAR(255),"
               "Fecha DATE,"
               "Articulo VARCHAR(255),"
               "Cantidad FLOAT,"
               "Precio FLOAT,"
               "Importe FLOAT"
               ")")

### The data file is loaded and read

In [ ]:
archivo_ventas = "data/ArticulosPanaderia.xlsx"

df = pd.read_excel(archivo_ventas)

registros = []
for _, row in df.iterrows():
    familia = row['FAMILIA']
    tipo = row['Tipo']
    fecha_venta = row['FechaVenta'].strftime('%Y-%m-%d')
    hora_venta = row['HoraVenta']
    articulo = row['Articulo']
    cantidad = row['Cantidad']
    precio = row['Precio']
    importe = row['Importe']
    registros.append((familia, tipo, fecha_venta, hora_venta, articulo, cantidad, precio, importe))

consulta = f"INSERT INTO {tabla_ventas} (FAMILIA, Tipo, FechaVenta, HoraVenta, Articulo, Cantidad, Precio, Importe) " \
           "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

In [ ]:
archivo_calendario = "data/Calendario.xlsx"

df = pd.read_excel(archivo_calendario)

registros = []
for _, row in df.iterrows():
    fecha = row['Fecha'].strftime('%Y-%m-%d')
    festivo = row['Festivo']
    registros.append((fecha,festivo))

consulta = f"INSERT INTO {tabla_calendario} (Fecha, Festivo) " \
           "VALUES (%s, %s)"

tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

In [ ]:
archivo_pedidos = "data/CantidadPedida.xlsx"

df = pd.read_excel(archivo_pedidos)

registros = []
for _, row in df.iterrows():
    tipo = row['Tipo']
    fecha_venta = row['Fecha'].strftime('%Y-%m-%d')
    articulo = row['Articulo']
    cantidad = row['Cantidad']
    precio = row['Precio']
    importe = row['Importe']
    registros.append(( tipo, fecha_venta, articulo, cantidad, precio, importe))

consulta = f"INSERT INTO {tabla_pedidos} (Tipo, Fecha, Articulo, Cantidad, Precio, Importe) " \
           "VALUES (%s, %s, %s, %s, %s, %s)"

tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

-----------